In [4]:
from PIL import Image
import torchvision.transforms as transforms
import cv2
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn.functional as f
import torchmetrics.functional as metrics
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, Subset

C:\Users\kdp\anaconda3\envs\Pytorch38\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 지정된 프로시저를 찾을 수 없습니다'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
#PreProcessing
preprocessing = transforms.Compose([transforms.Resize((32,32)), transforms.ToTensor()])

In [6]:
#폴뎌명이 분류 클래스가 되는 데이터셋 만들기
trainroot = 'datas/mid_dogs/'
dataset = ImageFolder(root=trainroot, transform=preprocessing) #전처리
loader = DataLoader(dataset=dataset, batch_size=10, shuffle=True) #데이터로더

In [7]:
# 데이터셋을 트레인셋과 테스트셋으로 나눔
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset = Subset(dataset, range(0, train_size))
test_dataset = Subset(dataset, range(train_size, len(dataset)))

# 트레인셋과 테스트셋에 대한 DataLoader 각각 생성
train_loader = DataLoader(dataset=train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=10, shuffle=False)

In [8]:
num_classes = len(dataset.classes)

In [9]:
# 이미지 / 라벨 분리
images, labels = [], []


In [10]:
for img, lab in dataset:
    images.append(np.array(img))
    labels.append(lab)


images = np.array(images)
labels = np.array(labels)

In [11]:
import torch.nn.functional as f

In [12]:
class mid_dogs(nn.Module):
    def __init__(self):
        super().__init__()
        
        #합성곱 계층 정의하기
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        
        #전결합 계층 정의하기
        self.fc1 = nn.Linear(64 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, num_classes)  # 모델의 출력 크기를 클래스 수에 맞게 조정
        
    def forward(self, x):
        # 합성곱 계층 연산
        x = f.relu(self.conv1(x))
        x = f.max_pool2d(x, 2, 2)  # 2x2 max pooling
        x = f.relu(self.conv2(x))
        x = f.max_pool2d(x, 2, 2)
        x = f.relu(self.conv3(x))
        x = f.max_pool2d(x, 2, 2)
    
        # 전결합 계층 연산
        x = x.view(-1, 64 * 4 * 4)  # batch size 유지하면서 flatten
        x = f.relu(self.fc1(x))
        x = self.fc2(x)
    
        return x


In [13]:
#인스턴스 생성
model = mid_dogs()

#DEVICE

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 손실함수
C = nn.CrossEntropyLoss()
OPTIMIZER = torch.optim.Adam(model.parameters(), lr=0.001)

#에포크Z
EPOCHS =100


In [14]:
def training(dataloader):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        OPTIMIZER.zero_grad()
        outputs = model(images)
        labels = labels.long()
        loss = C(outputs, labels)
        loss.backward()
        OPTIMIZER.step()
        running_loss += loss.item()
        
        
    return running_loss/len(dataloader) # 평균 손실 반환

In [15]:
training(train_loader)

3.39344587548059

In [16]:
def testing(dataloader):
    model.eval()
    with torch.no_grad():
        val_loss = []
        for cnt, (feature, target) in enumerate(dataloader):
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            target = target.squeeze()
            
            pre_target = model(feature)
            
            loss = C(pre_target, target.long())
            val_loss.append(loss)
            
        acc = metrics.accuracy(pre_target, target, task='multiclass', num_classes=num_classes)
        print(f'[VALID_LOSS] = {loss}\n\n[ACCURACY] = {acc.item()}')
        return val_loss

In [17]:
testing(test_loader)

[VALID_LOSS] = 10.961725234985352

[ACCURACY] = 0.0


[tensor(4.5734),
 tensor(4.6045),
 tensor(4.3933),
 tensor(4.5033),
 tensor(4.5374),
 tensor(4.3778),
 tensor(4.4140),
 tensor(4.4689),
 tensor(5.4514),
 tensor(11.2467),
 tensor(13.0104),
 tensor(11.6972),
 tensor(12.1407),
 tensor(12.8456),
 tensor(11.4671),
 tensor(12.8446),
 tensor(11.2664),
 tensor(13.0564),
 tensor(13.4563),
 tensor(13.5229),
 tensor(12.1563),
 tensor(11.4499),
 tensor(13.1345),
 tensor(12.5368),
 tensor(10.9902),
 tensor(12.3911),
 tensor(11.1229),
 tensor(12.0682),
 tensor(10.9107),
 tensor(10.6646),
 tensor(10.8259),
 tensor(10.4210),
 tensor(10.9867),
 tensor(11.2499),
 tensor(10.3919),
 tensor(11.4286),
 tensor(10.1015),
 tensor(10.4115),
 tensor(10.5127),
 tensor(10.7930),
 tensor(10.3829),
 tensor(11.8688),
 tensor(11.1725),
 tensor(12.4332),
 tensor(11.4826),
 tensor(10.9843),
 tensor(11.1876),
 tensor(10.9562),
 tensor(11.2083),
 tensor(10.8253),
 tensor(11.3523),
 tensor(11.6193),
 tensor(10.8500),
 tensor(12.3253),
 tensor(10.8647),
 tensor(11.2517),
 

In [18]:
def early_stopping(validation_losses, patience=3):
    if len(validation_losses) < patience + 1:
        return False  # 최소 횟수의 검증 손실 기록이 없는 경우 훈련 계속
    else:
        for i in range(1, patience + 1):
            if validation_losses[-i] < validation_losses[-(i+1)]:
                return False  # 이전 손실보다 현재 손실이 더 작으면 계속 훈련
        return True  # 향상되지 않은 횟수가 patience를 초과하면 훈련 중단

In [19]:
train_losses = []
test_losses = []
patience = 3  # 향상되지 않은 횟수

for epoch in range(EPOCHS):
    train_loss = training(train_loader)
    test_loss = testing(test_loader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    #print(f'[{epoch+1}/{EPOCHS}]\n\n TRAIN: {train_loss}, VALID: {test_loss}')
    
    if epoch >= patience:  # 최소 횟수 이상의 에포크에 대해
        validation_losses = test_losses[-(patience + 1):]  # 최근 patience+1 개의 검증 손실을 가져옴
        if early_stopping(validation_losses):  # 훈련 중단 조건 충족 여부 확인
            print(f"{epoch}th : Early stopping!")
            torch.save(model.state_dict(),'middledogs_model.pth')
            break

train_loss_avg = sum(train_losses).item() / len(train_losses)
test_loss_avg = sum(test_losses).item() / len(test_losses)
print(f'\n\n TRAIN: {train_loss_avg}, VALID: {test_loss_avg}')

[VALID_LOSS] = 12.357385635375977

[ACCURACY] = 0.0
[VALID_LOSS] = 11.733083724975586

[ACCURACY] = 0.0
[VALID_LOSS] = 10.485595703125

[ACCURACY] = 0.0
[VALID_LOSS] = 13.01984977722168

[ACCURACY] = 0.0
[VALID_LOSS] = 18.479141235351562

[ACCURACY] = 0.0
[VALID_LOSS] = 14.626815795898438

[ACCURACY] = 0.0
[VALID_LOSS] = 15.983972549438477

[ACCURACY] = 0.0
[VALID_LOSS] = 16.41154670715332

[ACCURACY] = 0.0
[VALID_LOSS] = 16.89019012451172

[ACCURACY] = 0.0
[VALID_LOSS] = 18.150793075561523

[ACCURACY] = 0.0
[VALID_LOSS] = 17.94178581237793

[ACCURACY] = 0.0
[VALID_LOSS] = 20.06035614013672

[ACCURACY] = 0.0
[VALID_LOSS] = 22.34174346923828

[ACCURACY] = 0.0
[VALID_LOSS] = 23.11543083190918

[ACCURACY] = 0.0
[VALID_LOSS] = 25.658649444580078

[ACCURACY] = 0.0
[VALID_LOSS] = 25.59210777282715

[ACCURACY] = 0.0
[VALID_LOSS] = 31.468544006347656

[ACCURACY] = 0.0
16th : Early stopping!


AttributeError: 'float' object has no attribute 'item'

In [ ]:
model = model
model.load_state_dict(torch.load('middledogs_model.pth'))
model.eval()